Install Pygame

In [2]:
%pip install pygame

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 1.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


#### -- [INIT] -- 
***

##### **SETUP** : Import Libraries and Init Globals
- `on_awake   ` (init_game => screen, refresh, load assets ... etc)
- `on_update  ` (game_loop => handle events, input, draws ..etc )
- `on_exit    ` (close_app => clear memory and shut down cpu/sys ... etc)

In [1]:
# `numpy` library random which is FASTER for generating large numbers of random values
from numpy import random
from typing import NamedTuple
import pygame
import sys

# Define the screen size
SCREEN_WIDTH, SCREEN_HEIGHT = 320, 240
FPS = 5
BACKGROUND_COLOR = (0, 0, 0)

class SystemContext (NamedTuple):
    screen :pygame.Surface
    clock  :pygame.time.Clock

class BaseEntity:
    def __init__(self):
        pass
        
    def update(self):
        pass

class DrawEntity(BaseEntity):
    def __init__(self, x,y, image):
        super().__init__()
        self.x = x
        self.y = y
        self.image = image

    def draw(self, screen : pygame.Surface):
        screen.blit(self.image, (self.x, self.y))

class GameEntity(DrawEntity):
    def __init__(self, x,y, image, scale = 1, rotation =0, speed = 0):
        super().__init__(x,y, image)
        self.vx = 0
        self.vy = 0
        self.rotation = rotation
        self.scale = scale
        self.speed = speed

    def update(self):
        super().update()
        # Update the position based on velocity
        self.x = (self.x + self.vx) % SCREEN_WIDTH
        self.y = (self.y + self.vy) % SCREEN_HEIGHT
        # Update the rotation
        self.rotation = (self.rotation + 1) % 360

def on_awake():
    ''' 
    Initialize pygame
    - SYS_SCREEN
    - SYS_CLOCK
    '''
    # Initialize pygame
    pygame.init()
    # Initialize the screen
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    # Initialize the clock
    # @audit-ok 💨 : Cap the frame rate else loop will run at system allows and 
    # # will lead to :
    # - high CPU usage
    # - inconsistent performance across different devices. 
    clock = pygame.time.Clock()

    # Set the title and icon (optional)
    pygame.display.set_caption('Asteroids_TON_Edition')
    icon = pygame.image.load('asteroid.png')
    pygame.display.set_icon(icon)
    return SystemContext(screen, clock)

def on_exit():
    ''' 
    Handle exit event
    '''
    pygame.quit()
    sys.exit()

# Game Loop
def on_update(context : SystemContext, asteroids : list):
    ''' 
    Game loop
    '''
    while True:
        try:
            # Set the frame rate
            context.clock.tick(FPS)
            # Event Handling
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    on_exit()
                if event.type == pygame.KEYUP:
                    if event.key == pygame.K_ESCAPE:
                        on_exit()
            # Update game state
            # Draw game objects
            # Update the display
            context.screen.fill(BACKGROUND_COLOR)
            for asteroid in asteroids:
                asteroid.update()
                asteroid.draw(context.screen)
            pygame.display.flip()
            
        except SystemExit:
            break

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


##### **CLASSES**

- Asteroids

In [2]:
class Asteroid(GameEntity):
    def __init__(self, x, y, image):
        super().__init__(x, y, image)
        self.vx = random.randint(-2, 2)
        self.vy = random.randint(-2, 2)
        self.rotation = random.randint(0, 359)

    def draw(self, screen : pygame.Surface):
        # Rotate the image
        rotated_image = pygame.transform.rotate(self.image, self.rotation)
        # Calculate the new image's rect to keep it centered
        rect = self.image.get_rect(center=self.image.get_rect(topleft=(self.x, self.y)).center)
        # Draw the asteroid
        screen.blit(rotated_image, rect.topleft)

#### -- [ON_AWAKE] --

In [3]:
context = on_awake()
asteroids = []
x = 160
y = 120
img = pygame.image.load('asteroid.png')
img = pygame.transform.scale(img, (32, 32))
asteroid = Asteroid(x, y, img)
asteroids.append(asteroid)
# @audit-ok 💨 : The memory and performance overhead of passing the `screen` 
# object to the function is MINIMAL, as it's just passing a REFERENCE to the 
# object.
on_update(context, asteroids)

#### -- [ON_UPDATE] --

#### -- [FOOTNOTES] --
***

Explain the `difference` between adding the `try and exception catching` above vs  
not doing a catch from `memory` and `performance` perspective
***

Adding a `try` and `except` block around your code introduces a minimal overhead  
in terms of memory and performance.  In general, the cost of using exception  
handling is low when no exceptions are raised.  

However, when an exception occurs : 
- the traceback needs to be generated  
- and the `interpreter has to search the exception hierarchy` to find the  
appropriate `except` block, which can be more `expensive`.

Here's a comparison between using exception handling and not using it in the  
context of the Pygame example we've discussed:

**Without exception handling:**

When you don't catch the `SystemExit` exception, the kernel running the Jupyter  
Notebook will display the message : 
- `An exception has occurred, use %tb to see the full traceback. SystemExit`  
whenever the game is terminated. 

This is because : 

- the `SystemExit` exception is raised, 
- caught by the notebook's kernel, 
- and displayed as output. 

The game loop will still exit as expected, and the exception handling overhead  
is minimal. However, you'll see the traceback message every time the game is  
closed, which might be `undesirable from a user experience standpoint`.

**With exception handling:**

When you include a `try` and `except` block to catch the `SystemExit` exception,  
you introduce a small overhead associated with the exception handling mechanism.  

However, this overhead is generally negligible in the context of a game loop.  
When the game is terminated, the `SystemExit` exception is raised and caught  
by the `except` block, allowing the loop to exit silently without displaying  
the traceback message. This can provide a cleaner user experience.

In summary, the difference in memory and performance between using exception  
handling and not using it in this case is minimal. The primary reason to include  
exception handling in this scenario is to improve the user experience by  
preventing the traceback message from being displayed when the game is terminated.

Explain the trade off between declaring variable as global vs passing as reference with respect to memory and performance
***

When deciding whether to use a global variable or pass a variable by reference, there are trade-offs to consider regarding memory, performance, and general programming practices.

**Global variables:**

Pros:

- Easy to access from any part of the code without needing to pass them as arguments.
- Can help reduce the number of arguments a function needs, making the function signature simpler.

Cons:

- Can lead to unexpected behavior and bugs, as any part of the code can modify the global variable.
- Makes code harder to understand and maintain, as it's not clear which functions rely on or modify the global variable.
- Less modular and harder to reuse functions in other programs, since they depend on the global variable.

**Passing by reference:**

Pros:

- Makes code more modular, as functions explicitly state in their signature what variables they need.
- Easier to understand, maintain, and reuse, as it's clear which functions use or modify the variable.
- Better encapsulation, as each function deals only with the data it needs.

Cons:

- Can increase the number of function arguments, making the function signature more complex.
- Slightly more complex to work with, as you need to pass the variable explicitly.

Regarding memory and performance, there is generally not a significant difference between using global variables and passing variables by reference. In both cases, the memory used for the variable is allocated once and accessed by reference, so there is no additional memory overhead. The performance depends on the specific use case and how well the code is optimized.

However, it is worth mentioning that using global variables can lead to less maintainable code, which may indirectly affect performance over time, as it becomes more difficult to optimize and debug the code.

In general, it's recommended to use local variables and pass them by reference whenever possible to promote modularity, maintainability, and readability. Only use global variables when you have a good reason to do so, such as when dealing with truly global state or constants.